  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo

### Getting started
Define the paths for the dataset and trained models in the `notebooks/config/UNetTraining.py` file.  

In [1]:
import os
os.environ["MKL_NUM_THREADS"] = '16'
os.environ["NUMEXPR_NUM_THREADS"] = '16'
os.environ["OMP_NUM_THREADS"] = '16'
print(os.environ.get('OMP_NUM_THREADS'))

16


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')

import imageio
import os

import time
import rasterio.warp             # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.UNet_572 import UNet  #
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision,F1_score
from tensorflow.keras.losses import BinaryCrossentropy as bce
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info_572 import FrameInfo
from core.dataset_generator_572 import DataGenerator
from core.visualize import display_images,plot
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pickle
import random

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon
#matplotlib.use("Agg")

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory.
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print(tf.__version__)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
2.9.0


In [3]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.device('/gpu:1')

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/UNetTraining.py)
# Please provide required info in the file before continuing with this notebook. 
# hbh: in this scene,a new config named UNetTraining_sequential is created to distinguish from the original
from config import UNetTraining_572
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import UNetTraining
config = UNetTraining_572.Configuration()

## 参数初始化

In [6]:
OPTIMIZER = adaDelta
OPTIMIZER =  mixed_precision.LossScaleOptimizer(OPTIMIZER)
OPTIMIZER_NAME = 'AdaDelta'

# OPTIMIZER = adam
# OPTIMIZER = mixed_precision.LossScaleOptimizer(OPTIMIZER)
# OPTIMIZER_NAME = 'adam'

In [7]:
# LOSS = tversky 
# LOSS_NAME = 'tversky'

# LOSS=focalTversky
# LOSS_NAME = 'focalTversky'

#LOSS=tf.keras.losses.BinaryCrossentropy()
#LOSS_NAME = 'bce'

# LOSS=bce_dice_loss
# LOSS_NAME = 'bce_dice_loss'

LOSS=dice_loss
LOSS_NAME = 'dice_loss'


In [8]:
def readImgs(path_to_write, fn):
    image = rasterio.open(os.path.join(path_to_write, fn))
#     print(fn)
    read_image = image.read()/1000
#     print(read_image[1].max())
    
    comb_img = np.transpose(read_image, axes=(1,2,0))
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(config.image_fn,config.annotation_fn).replace(config.image_type,config.ann_type)))
    annotation = np.array(annotation_im)
    rowNum=annotation.shape[0]/config.patch_size[0]#config.output_size[0]
    colNum=annotation.shape[1]/config.patch_size[1]#config.output_size[1]
#     if(read_image.shape[1]- annotation.shape[0]!=184) or(read_image.shape[2]- annotation.shape[1]!=184):
#                 print(fn)
    f = FrameInfo(comb_img, annotation)
    return f,rowNum*colNum
    
def readFrames(dataType):
    frames=[]
    numList=[]
    print(dataType)
#     for i in range(3,4):
    for i in range(0,config.type_num):#config.type_num
        path_to_write=os.path.join(config.dataset_dir,'{}/type{}'.format(dataType,i))
        all_files = os.listdir(path_to_write)
        all_files_image = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#image.png
        print('type{} image number:{}'.format(i,len(all_files_image)))
        for j, fn in enumerate(all_files_image):
            f,num = readImgs(path_to_write,fn)
            frames.append(f)
            numList.append(num)
    return frames,numList


## 数据集准备

In [9]:
frames,numList=readFrames('train')
percentages=np.array(numList)
train_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total training img count:'+str(len(frames)))
print('total training patches count:{}'.format(train_sum))
train_generator = DataGenerator(config.input_size, config.output_size,frames, augmenter = 'iaa').random_generator(config.BATCH_SIZE,percentages)#,normalize =

train
type0 image number:156
type1 image number:141
type2 image number:68
type3 image number:53
type4 image number:9
type5 image number:17
total training img count:444
total training patches count:7901.780909408757


In [ ]:
# numList=readNum('train')
# percentages=np.array(numList)
# print(percentages.sum())

In [10]:
frames,numList=readFrames('val')
percentages=np.array(numList)
val_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total validation img count:'+str(len(frames)))
print('total validation patches count:{}'.format(val_sum))
val_generator = DataGenerator(config.input_size, config.output_size, frames, augmenter = None).random_generator(config.BATCH_SIZE,percentages)#, normalize = config.normalize

val
type0 image number:53
type1 image number:48
type2 image number:24
type3 image number:18
type4 image number:3
type5 image number:5
total validation img count:151
total validation patches count:2595.4350977768136


In [12]:
del frames

In [ ]:
for _ in range(1):
    train_images, real_label = next(train_generator) 
    print(train_images.shape)
    print(real_label.shape)
    display_images(train_images,real_label,pad=92,output_size=config.output_size[0])

In [ ]:
for _ in range(1):
    val_images, val_label = next(val_generator) 
    print(val_images.shape)
    display_images(val_images,val_label,pad=92,output_size=config.output_size[0])

### 训练参数设置

In [13]:
timestr = time.strftime("%Y%m%d-%H%M")
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 

if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
model_name='_{}_{}_{}_{}_{}.h5'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs,config.input_shape[0])
model_path = os.path.join(config.model_path,'lakes'+model_name)
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 
print(model_path)

G:\lakemapping\5_saved_models\lakes_20240110-2311_AdaDelta_dice_loss_012345_572.h5


In [14]:
# Define the model and compile it  
print(config.input_shape)
shape=[config.BATCH_SIZE, *config.input_shape]
model = UNet([config.BATCH_SIZE, *config.input_shape],config.input_label_channel)
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy, recall, precision,F1_score, IoU])

(572, 572, 5)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 572, 572, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 570, 570, 64  2944        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 568, 568, 64  36928       ['conv2d[0][0]']                 
                                )                                               

 conv2d_12 (Conv2D)             (None, 102, 102, 25  1769728     ['concatenate_1[0][0]']          
                                6)                                                                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 100, 100, 25  590080      ['conv2d_12[0][0]']              
                                6)                                                                
                                                                                                  
 up_sampling2d_2 (UpSampling2D)  (None, 200, 200, 25  0          ['conv2d_13[0][0]']              
                                6)                                                                
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 200, 200, 25  1024       ['up_sampling2d_2[0][0]']        
 rmalizati

In [15]:
# Define callbacks      for the early stopping of training, LearningRateScheduler and model checkpointing 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea： It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience 个epoch, reduce by a factor of 0.33, new_lr = lr * factor). 
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced. 

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16) 

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=40)


log_dir = os.path.join('./logs','UNet'+model_name)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta, early

## 训练模型
保存history文件并绘图

In [16]:
loss_history = model.fit(train_generator, 
                         steps_per_epoch=train_sum//config.BATCH_SIZE,#config.MAX_TRAIN_STEPS,
                         epochs=config.NB_EPOCHS, 
                         validation_data=val_generator,
                         validation_steps=val_sum//config.BATCH_SIZE,#config.VALID_IMG_COUNT,
                         callbacks=callbacks_list,
                         workers=1,
                         # shuffle=True,
#                          use_multiprocessing=True # the generator is not very thread safe 
                         #max_queue_size = 60,
                        )
h=loss_history.history
with open('history_{}_{}_{}_{}_{}.txt'.format(timestr,OPTIMIZER_NAME,LOSS_NAME, chs,config.input_shape[0]), 'wb') as file_pi:
    pickle.dump(h, file_pi)
plot(h,timestr, OPTIMIZER_NAME,LOSS_NAME, config.input_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

Epoch 1/150
493/493 [==============================] - ETA: 0s - loss: 0.3132 - dice_loss: 0.3130 - accuracy: 0.9793 - recall: 0.7946 - precision: 0.6459 - F1_score: 0.6889 - IoU: 0.5645
Epoch 1: val_loss improved from inf to 0.26791, saving model to G:\lakemapping\5_saved_models\lakes_20240110-2311_AdaDelta_dice_loss_012345_572.h5
493/493 [==============================] - 367s 624ms/step - loss: 0.3132 - dice_loss: 0.3130 - accuracy: 0.9793 - recall: 0.7946 - precision: 0.6459 - F1_score: 0.6889 - IoU: 0.5645 - val_loss: 0.2679 - val_dice_loss: 0.2677 - val_accuracy: 0.9847 - val_recall: 0.7363 - val_precision: 0.7621 - val_F1_score: 0.7323 - val_IoU: 0.5988
Epoch 2/150
493/493 [==============================] - ETA: 0s - loss: 0.2193 - dice_loss: 0.2192 - accuracy: 0.9872 - recall: 0.8832 - precision: 0.7281 - F1_score: 0.7810 - IoU: 0.6630
Epoch 2: val_loss improved from 0.26791 to 0.23800, saving model to G:\lakemapping\5_saved_models\lakes_20240110-2311_AdaDelta_dice_loss_012345_

Epoch 14/150
493/493 [==============================] - ETA: 0s - loss: 0.1740 - dice_loss: 0.1738 - accuracy: 0.9903 - recall: 0.8783 - precision: 0.8007 - F1_score: 0.8263 - IoU: 0.7198
Epoch 14: val_loss did not improve from 0.16827
493/493 [==============================] - 304s 617ms/step - loss: 0.1740 - dice_loss: 0.1738 - accuracy: 0.9903 - recall: 0.8783 - precision: 0.8007 - F1_score: 0.8263 - IoU: 0.7198 - val_loss: 0.1728 - val_dice_loss: 0.1727 - val_accuracy: 0.9901 - val_recall: 0.9024 - val_precision: 0.7779 - val_F1_score: 0.8274 - val_IoU: 0.7221
Epoch 15/150
493/493 [==============================] - ETA: 0s - loss: 0.1694 - dice_loss: 0.1692 - accuracy: 0.9911 - recall: 0.8810 - precision: 0.8085 - F1_score: 0.8309 - IoU: 0.7277
Epoch 15: val_loss did not improve from 0.16827
493/493 [==============================] - 303s 613ms/step - loss: 0.1694 - dice_loss: 0.1692 - accuracy: 0.9911 - recall: 0.8810 - precision: 0.8085 - F1_score: 0.8309 - IoU: 0.7277 - val_loss

Epoch 28/150
493/493 [==============================] - ETA: 0s - loss: 0.1069 - dice_loss: 0.1068 - accuracy: 0.9938 - recall: 0.9007 - precision: 0.8962 - F1_score: 0.8933 - IoU: 0.8143
Epoch 28: val_loss improved from 0.11339 to 0.11243, saving model to G:\lakemapping\5_saved_models\lakes_20240110-2311_AdaDelta_dice_loss_012345_572.h5
493/493 [==============================] - 311s 629ms/step - loss: 0.1069 - dice_loss: 0.1068 - accuracy: 0.9938 - recall: 0.9007 - precision: 0.8962 - F1_score: 0.8933 - IoU: 0.8143 - val_loss: 0.1124 - val_dice_loss: 0.1123 - val_accuracy: 0.9939 - val_recall: 0.9132 - val_precision: 0.8744 - val_F1_score: 0.8877 - val_IoU: 0.8099
Epoch 29/150
493/493 [==============================] - ETA: 0s - loss: 0.1008 - dice_loss: 0.1007 - accuracy: 0.9947 - recall: 0.9060 - precision: 0.9038 - F1_score: 0.8993 - IoU: 0.8251
Epoch 29: val_loss improved from 0.11243 to 0.10316, saving model to G:\lakemapping\5_saved_models\lakes_20240110-2311_AdaDelta_dice_loss

Epoch 42/150
493/493 [==============================] - ETA: 0s - loss: 0.0824 - dice_loss: 0.0823 - accuracy: 0.9956 - recall: 0.9209 - precision: 0.9224 - F1_score: 0.9177 - IoU: 0.8546
Epoch 42: val_loss did not improve from 0.09354
493/493 [==============================] - 306s 619ms/step - loss: 0.0824 - dice_loss: 0.0823 - accuracy: 0.9956 - recall: 0.9209 - precision: 0.9224 - F1_score: 0.9177 - IoU: 0.8546 - val_loss: 0.0951 - val_dice_loss: 0.0950 - val_accuracy: 0.9950 - val_recall: 0.9064 - val_precision: 0.9112 - val_F1_score: 0.9051 - val_IoU: 0.8361
Epoch 43/150
493/493 [==============================] - ETA: 0s - loss: 0.0862 - dice_loss: 0.0861 - accuracy: 0.9955 - recall: 0.9185 - precision: 0.9183 - F1_score: 0.9139 - IoU: 0.8483
Epoch 43: val_loss did not improve from 0.09354
493/493 [==============================] - 306s 620ms/step - loss: 0.0862 - dice_loss: 0.0861 - accuracy: 0.9955 - recall: 0.9185 - precision: 0.9183 - F1_score: 0.9139 - IoU: 0.8483 - val_loss

493/493 [==============================] - ETA: 0s - loss: 0.0711 - dice_loss: 0.0710 - accuracy: 0.9963 - recall: 0.9296 - precision: 0.9335 - F1_score: 0.9290 - IoU: 0.8712
Epoch 70: val_loss did not improve from 0.08681
493/493 [==============================] - 305s 618ms/step - loss: 0.0711 - dice_loss: 0.0710 - accuracy: 0.9963 - recall: 0.9296 - precision: 0.9335 - F1_score: 0.9290 - IoU: 0.8712 - val_loss: 0.0917 - val_dice_loss: 0.0916 - val_accuracy: 0.9947 - val_recall: 0.8809 - val_precision: 0.9483 - val_F1_score: 0.9085 - val_IoU: 0.8413
Epoch 71/150
493/493 [==============================] - ETA: 0s - loss: 0.0710 - dice_loss: 0.0709 - accuracy: 0.9963 - recall: 0.9290 - precision: 0.9341 - F1_score: 0.9291 - IoU: 0.8719
Epoch 71: val_loss did not improve from 0.08681
493/493 [==============================] - 306s 620ms/step - loss: 0.0710 - dice_loss: 0.0709 - accuracy: 0.9963 - recall: 0.9290 - precision: 0.9341 - F1_score: 0.9291 - IoU: 0.8719 - val_loss: 0.1112 - va

493/493 [==============================] - ETA: 0s - loss: 0.0592 - dice_loss: 0.0592 - accuracy: 0.9969 - recall: 0.9418 - precision: 0.9427 - F1_score: 0.9409 - IoU: 0.8908
Epoch 98: val_loss did not improve from 0.08383
493/493 [==============================] - 306s 620ms/step - loss: 0.0592 - dice_loss: 0.0592 - accuracy: 0.9969 - recall: 0.9418 - precision: 0.9427 - F1_score: 0.9409 - IoU: 0.8908 - val_loss: 0.0890 - val_dice_loss: 0.0889 - val_accuracy: 0.9951 - val_recall: 0.9075 - val_precision: 0.9256 - val_F1_score: 0.9111 - val_IoU: 0.8454
Epoch 99/150
493/493 [==============================] - ETA: 0s - loss: 0.0573 - dice_loss: 0.0573 - accuracy: 0.9970 - recall: 0.9451 - precision: 0.9438 - F1_score: 0.9428 - IoU: 0.8946
Epoch 99: val_loss did not improve from 0.08383
493/493 [==============================] - 306s 621ms/step - loss: 0.0573 - dice_loss: 0.0573 - accuracy: 0.9970 - recall: 0.9451 - precision: 0.9438 - F1_score: 0.9428 - IoU: 0.8946 - val_loss: 0.0842 - va

493/493 [==============================] - ETA: 0s - loss: 0.0504 - dice_loss: 0.0503 - accuracy: 0.9972 - recall: 0.9496 - precision: 0.9517 - F1_score: 0.9497 - IoU: 0.9058
Epoch 126: val_loss did not improve from 0.07844
493/493 [==============================] - 307s 622ms/step - loss: 0.0504 - dice_loss: 0.0503 - accuracy: 0.9972 - recall: 0.9496 - precision: 0.9517 - F1_score: 0.9497 - IoU: 0.9058 - val_loss: 0.0932 - val_dice_loss: 0.0932 - val_accuracy: 0.9948 - val_recall: 0.8848 - val_precision: 0.9395 - val_F1_score: 0.9069 - val_IoU: 0.8373
Epoch 127/150
493/493 [==============================] - ETA: 0s - loss: 0.0547 - dice_loss: 0.0546 - accuracy: 0.9971 - recall: 0.9426 - precision: 0.9508 - F1_score: 0.9454 - IoU: 0.8989
Epoch 127: val_loss did not improve from 0.07844
493/493 [==============================] - 308s 623ms/step - loss: 0.0547 - dice_loss: 0.0546 - accuracy: 0.9971 - recall: 0.9426 - precision: 0.9508 - F1_score: 0.9454 - IoU: 0.8989 - val_loss: 0.0835 -

In [ ]:
##### 读取现有history文件
with open('.txt','rb')as file_pi: 
    h=pickle.load(file_pi)
print(h)

In [17]:
plot(h, OPTIMIZER_NAME,LOSS_NAME, config.patch_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

## 模型精度评价

In [13]:
# Load model after training 
model_path=os.path.join(config.model_path,'lakes_20240110-2311_AdaDelta_dice_loss_012345_572.h5')
model = load_model(model_path, custom_objects={'dice loss': LOSS, 'accuracy':accuracy ,'recall':recall, 'F1_score':F1_score,'precision':precision,'IoU': IoU}, compile=False) 
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,F1_score, precision, IoU])

In [10]:
# # Print one batch on the training/test data! 
# for i in range(1):
#     test_images, real_label = next(test_generator)
#     #3 images per row: GSW, label, prediction
#     prediction = model.predict(test_images, steps=1)
#     prediction[prediction>0.5]=1
#     prediction[prediction<=0.5]=0
#     display_images(np.concatenate((test_images, real_label, prediction), axis = -1))# test_images( NDWI), real_label(label), prediction

In [ ]:
j=1
frames=[]
path_to_write=os.path.join(config.dataset_dir,'test/type'+str(j))
all_files = os.listdir(path_to_write)
all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
for i, fn in enumerate(all_image_files):
    f,nums = readImgs(path_to_write,fn)
    frames.append(f)
test_DGT=DataGenerator(config.input_size, config.output_size,frames, augmenter = None)
tp_img,tp_ann= test_DGT.all_sequential_patches(config.step_size)
print(len(tp_ann))

titles=['ndwi','rgb','swir','annotation','prediction']
for i in range(len(tp_ann//config.BATCH_SIZE)+1):
    prediction = model.predict(tp_img[i*config.BATCH_SIZE:(i+1)*config.BATCH_SIZE], steps=1)
    prediction[prediction>0.5]=1
    prediction[prediction<=0.5]=0
    fn=os.path.join(config.base_dir,r'4_prediction\sample746\image\type{}_{}_{}.png'.format(j,i,config.output_size[0]))
    display_images(tp_img[i*config.BATCH_SIZE:(i+1)*config.BATCH_SIZE], tp_ann[i*config.BATCH_SIZE:(i+1)*config.BATCH_SIZE], prediction,92,config.output_size[0],fn.format(i),titles=titles)

## 总体精度评价

In [10]:
frames,numList=readFrames('test')
test_patches = DataGenerator(config.input_size, config.output_size, frames, augmenter = None).all_sequential_patches(config.step_size)
print('test patchs number:',len(test_patches[0]))

test
type0 image number:53
type1 image number:48
type2 image number:23
type3 image number:18
type4 image number:3
type5 image number:6
test patchs number: 5254


In [ ]:
ev=model.evaluate(test_patches[0],test_patches[1],config.BATCH_SIZE)
ev.append(len(test_patches[0]))

In [19]:
print(ev)

[0.11038431525230408, 0.11386916786432266, 0.9946683049201965, 0.9123051166534424, 0.8933606743812561, 0.8861443400382996, 0.8190804719924927]


## 分类别精度评价

In [ ]:
# ev_list=[]
for i in range(0,config.type_num):
    frames=[]
    test_patchs_type=[]
    path_to_write=os.path.join(config.dataset_dir,'test/type'+str(i))
    all_files = os.listdir(path_to_write)
    all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
    for j, fn in enumerate(all_image_files):
        f ,num = readImgs(path_to_write,fn)
        frames.append(f)
    test_patchs_type = DataGenerator(config.input_size, config.output_size, frames, augmenter = None).all_sequential_patches(config.step_size)
#     print('type{} patches number:{}'.format(i,len(test_patchs_type[0])))
    ev2=model.evaluate(test_patchs_type[0],test_patchs_type[1],config.BATCH_SIZE,verbose=1)
    ev2.append(len(test_patchs_type[0]))
    ev_list.append(ev2)

In [23]:
print('      patch_num   loss  dice_loss accuracy   recall  precision F1_score     IoU')
# print('total: {:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(ev[7],ev[0],ev[1],ev[2],ev[3],ev[4],ev[5],ev[6]))
for i in range(0,config.type_num):
    print('type{}:{:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(i,ev_list[i][7],ev_list[i][0],ev_list[i][1],ev_list[i][2],ev_list[i][3],ev_list[i][4],ev_list[i][5],ev_list[i][6]))

        loss diclo accur recal preci F1_s IoU
total: 0.184 0.239 0.996 0.717 0.932 0.761 0.657
type0: 0.068 0.068 0.997 0.929 0.946 0.932 0.879
type1: 0.121 0.122 0.993 0.922 0.863 0.878 0.804
type2: 0.111 0.111 0.998 0.908 0.886 0.889 0.810
type3: 0.252 0.251 0.987 0.914 0.718 0.749 0.672
type4: 0.097 0.097 0.989 0.874 0.936 0.903 0.824
type5: 0.184 0.239 0.996 0.717 0.932 0.761 0.657
